# Prompt + LLM
基本构成 
PropmtTemplate/ChatPromptTemplate --> LLM/ChatModel --> OutputParser

In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.chat_models import ChatOllama

prompt = ChatPromptTemplate.from_template("讲一个关于{foo}的笑话")
model = ChatOllama(model="qwen2.5:7b")
chain = prompt | model
for s in chain.stream({"foo":"Faker"}):
  print(s.content, end="", flush=True)

当然可以，这里有一个关于Faker（《英雄联盟》中的知名职业选手）的轻松幽默小故事：

为什么Faker不养宠物龙？

因为他的口袋里总是装着“治疗”和“闪现”的快捷方式，没有地方给龙留爪子了！

这个笑话是基于大家对Faker在游戏中的操作技术非常熟练的印象而创造的一个简单幽默，实际上这只是个玩笑，并非事实。希望你喜欢！

## 自定义停止输出符号

In [6]:
chain = prompt | model.bind(stop=['\n'])
chain.invoke({"foo":"Faker"}).content

'当然，这里有一个关于《英雄联盟》职业选手Faker的幽默小故事：'

## 输出解析器

In [10]:
from langchain_core.output_parsers import StrOutputParser
chain = prompt | model | StrOutputParser()
chain.invoke({"foo":"女人"})

'当然，这里有一个轻松幽默的小笑话，希望你会喜欢：\n\n为什么2008年是个非常特别的一年？\n因为“8”这个数字对很多人来说象征着财源广进，而在那一年里，“她”（谐音）来了。\n\n请注意，这只是一个玩笑，并带有一定的地域性和文化性，可能在不同的地方有不同的解读方式。希望你能够会心一笑！'

## 与函数的混合使用

In [ ]:
# from langchain.output_parsers.openai_functions import JsonOutputFunctioonParser
# chain = (
#   prompt
#   | model.bind(function_call={"name":"joke"}, functions=functions)
#   | JsonOutputFunctioonParser()
# )

## 使用Runnables来连接多链结构

In [9]:
# 使用 itemgetter 提取特定键的值
from operator import itemgetter

from langchain.schema import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.chat_models import ChatOllama

prompt1 = ChatPromptTemplate.from_template(
  "{person}来自于哪个城市？"
)

prompt2 = ChatPromptTemplate.from_template(
  "{city}属于哪个省？ 用{language}回答"
)

model = ChatOllama(model="qwen2.5:7b")

chain1 = prompt1 | model | StrOutputParser()

# 使用 chain1 的输出作为 city，并通过 itemgetter 提取 language 键的值
# 构建一个新的链 chain2，该链将 city 和 language 作为输入传递给 prompt2
chain2 = (
  {
    "city": chain1,  # 从 chain1 获取城市信息
    "language": itemgetter("language")  # 提取输入中的 language 键值
  }
  | prompt2  # 使用 prompt2 模板生成新的提示
  | model  # 使用模型生成响应
  | StrOutputParser()  # 解析模型的输出为字符串
)

chain1.invoke({"person":"马化腾"})


'马化腾出生于广东省东莞市，但他更多被人熟知的是与腾讯公司的紧密联系。腾讯公司总部位于深圳市南山区，但马化腾的个人经历和职业生涯与中国多个城市都有紧密的关系。'

In [10]:

chain2.invoke({"person":"马化腾", "language":"zh-cn"})

'马化腾来自广东省。'

---

In [11]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser

model = ChatOllama(model="qwen2.5:7b")

prompt1 =  ChatPromptTemplate.from_template(
  "生成一个{attribute}属性的颜色，除了返回这个颜色的名字不做其他的事情:"
)
prompt2 =  ChatPromptTemplate.from_template(
  "什么水果是这个颜色:{color}，除了返回这个水果的名字不做其他的事情:"
)
prompt3 =  ChatPromptTemplate.from_template(
  "哪个国家的国旗有这个颜色:{color}，除了返回这个国家的名字不做其他的事情:"
)
prompt4 =  ChatPromptTemplate.from_template(
  "有这个颜色的水果是:{fruit}, 有这个颜色的国旗是{country}"
)

model_parser = model | StrOutputParser()


In [12]:
# 生成一个颜色
# 创建一个颜色生成器链条
# 1. 使用 RunnablePassthrough 将输入的属性直接传递给 prompt1 模板。
# 2. prompt1 模板生成一个关于颜色属性的提示。
# 3. 使用 model_parser（模型 + 输出解析器）解析生成的颜色名称。
color_generator = (
  {"attribute": RunnablePassthrough()}  # 直接传递输入的属性
  | prompt1  # 使用 prompt1 模板生成提示
  | {"color": model_parser}  # 使用模型和解析器生成颜色名称
)

# 定义一个从颜色到水果的映射链条
# 1. 使用 prompt2 模板生成一个关于颜色对应水果的提示。
# 2. 使用 model_parser（模型 + 输出解析器）解析生成的水果名称。
color2fruit = prompt2 | model_parser

# 定义一个从颜色到国家的映射链条
# 1. 使用 prompt3 模板生成一个关于颜色对应国家的提示。
# 2. 使用 model_parser（模型 + 输出解析器）解析生成的国家名称。
color2country = prompt3 | model_parser

# 创建一个问题生成器链条
# 1. 使用 color_generator 生成颜色。
# 2. 将生成的颜色作为输入，分别传递给 color2fruit 和 color2country 链条。
# 3. color2fruit 生成对应颜色的水果名称，color2country 生成对应颜色的国家名称。
question_generator = (
  color_generator
  |  # 将颜色生成器的输出传递给下游链条
  {"fruit": color2fruit, "country": color2country}  # 同时生成水果和国家信息
)


In [13]:
question_generator.invoke("强力的")

{'fruit': '火焰般红的草莓。',
 'country': '没有国家的国旗主色调是“炽热之炎”这种描述。如果你是指红色，那么中国和越南的国旗是主要为红色。但请注意，“炽热之炎”并不是一个标准的颜色名称或代码，在实际的国旗颜色中并不存在这样的描述。\n\n请确认你的问题是否准确，如果需要查询特定颜色对应国家的国旗，请提供更具体的信息或者正确的颜色名称。'}

## 多链执行与结果合并
```mermaid
graph TD;
  input-->分支1;
  input-->分支2;
  input-->分支3;
  分支1-->合并结束;
  分支2-->合并结束;
  分支3-->合并结束;
```

# 唯物辩证链

In [4]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser

model = ChatOllama(model="qwen2.5:7b", temperature=0)

prompt =  ChatPromptTemplate.from_template(
  "生成一个关于{input}的论点:"
)
prompt1 =  ChatPromptTemplate.from_template(
  "列出以下内容的优点或者积极方面:{base_response}:"
)
prompt2 =  ChatPromptTemplate.from_template(
  "列出以下内容的缺点或消极方面:{base_response}:"
)
prompt_final = ChatPromptTemplate.from_messages(
  [
    ("ai", "{original_response}"),
    ("human","积极: \n{result_good}\n\n     消极\n\n{result_bad}"),
    ("system", "根据评论生成最终回复")
  ]
)

chain_question = prompt | model | StrOutputParser() | {"base_response": RunnablePassthrough()}
arguments_for = prompt1 | model | StrOutputParser()
arguments_against = prompt2 | model | StrOutputParser()
final_responder = prompt_final | model | StrOutputParser()

chain = (
  chain_question |
  {"base_response": RunnablePassthrough()} # Pass through the intermediate output
  |
  {
    "result_good": arguments_for,
    "result_bad" : arguments_against,
    "original_response": itemgetter("base_response"),
    "base_response": RunnablePassthrough(),  # Pass through the intermediate output
  } |
  final_responder
)

In [6]:
# chain.invoke({"input":"生孩子"})

for s in chain.stream({"input":"年轻人不生孩子"}):
  print(s, end="", flush=True)


根据评论生成最终回复：

在讨论年轻人选择不生育的问题时，可以从多个角度来构建论点。这里提供一种可能的视角：

### 积极方面

1. **经济负担减少**：
   - 减轻了因养育孩子而产生的高额教育费用、住房成本以及长期医疗和日常开销。

2. **职业发展与个人追求优先**：
   - 专注于提升自我竞争力，实现个人目标或追求兴趣爱好，避免生育可能带来的生活约束。

3. **生活方式的自由选择**：
   - 更加灵活地安排个人时间，保持个人兴趣和社交活动，享受更加自由的生活方式。

4. **应对环境挑战的能力增强**：
   - 在面对全球气候变化等环境问题时，能够更好地规划未来家庭，减少育儿风险。

5. **社会支持系统的利用**：
   - 通过充分利用现有的社会支持体系（如托儿服务、育儿假政策），提高生活质量并减轻育儿压力。

### 消极方面

1. **经济负担增加**
   - 教育成本：随着教育费用的不断上涨，父母可能需要为孩子的学费、课外辅导班等支付高昂的费用。
   - 住房问题：大城市的生活成本高企，年轻人往往需要较长时间才能积累足够的经济实力来购买或租赁合适的住所。这可能导致他们推迟生育计划，甚至放弃生育。
   - 医疗与生活保障：养育孩子所需的长期医疗和日常开销也是不容忽视的负担。

2. **职业发展与个人追求**
   - 职业竞争激烈：现代社会中，职场竞争异常激烈，年轻人往往需要投入大量时间和精力来提升自我竞争力。这可能会导致他们将更多的时间和资源投入到事业上，而减少了对生育的关注。
   - 个人兴趣与梦想：一些年轻人可能更倾向于追求自己的兴趣爱好或实现个人目标，认为生育会限制他们的发展空间。

3. **生活方式选择**
   - 生活方式的转变：随着社会观念的变化，越来越多的年轻人开始追求更加自由、灵活的生活方式。养育孩子可能会带来一定的生活约束。
   - 家庭与社交平衡：在快节奏的社会中，保持个人兴趣和社交活动同样重要，生育可能会影响这部分时间分配。

4. **环境因素**
   - 环境变化：全球气候变化等环境问题也可能影响到年轻人对未来家庭的规划。例如，极端天气事件频发可能会增加育儿风险。
   - 社会支持系统不足：在一些地区或国家，缺乏足够的社会支持体系（如托儿服务、育儿假政策）也会影响年轻人的生育

### 查询SQL

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser

# 定义一个用于生成 SQL 查询的提示模板
sql_prompt = ChatPromptTemplate.from_template(
  "根据以下需求生成一个SQL查询：{query_description}"
)

# 创建一个模型实例
model = ChatOllama(model="qwen2.5:7b", temperature=0)

# 将提示模板与模型和输出解析器组合成一个链
sql_chain = sql_prompt | model | StrOutputParser()

# 示例输入：描述查询需求
query_description = "查询名为 employees 的表中所有年龄大于30岁的员工的姓名和职位。"

# 执行链并生成 SQL 查询
sql_query = sql_chain.invoke({"query_description": query_description}).content
print(sql_query)

### 生成Python代码 自定义输出解析器

In [ ]:
! pip install langchain_experimental

In [26]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_experimental.utilities import PythonREPL

# 定义一个用于生成 Python 代码的提示模板
template = """根据用户需求帮助用户编写Python代码, 只需返回markdown格式的Python代码, 比如:
  ```python
  ...
  ```
  """
python_code_prompt = ChatPromptTemplate.from_messages(
  [
    ("system",template),
    ("human","{input}")
  ]
)

# 创建一个模型实例
model = ChatOllama(model="qwen2.5:7b", temperature=0)


# 定义一个函数，用于生成并执行代码
def _sanitize_output(text: str) -> str:
  """
  一个帮助函数，用于清理模型生成的输出。
  该函数接收一个字符串输入，提取包含在三重反引号（```python ... ```) 中的Python代码块，
  并仅返回代码内容。
  参数:
  - text (str): 包含模型输出的输入字符串，其中包括一个Python代码块。
  返回值:
  - str: 从输入字符串中提取的Python代码。
  """
  # 使用第一次出现的 "```python" 将输入文本分为两部分。
  # 第一部分（在 "```python" 之前）被丢弃，第二部分（在 "```python" 之后）被保留。
  _, after = text.split("```python", 1)

  # 进一步使用第一次出现的 "```" 分割剩余文本。
  # 这将从任何尾随内容中隔离出Python代码块。
  code = after.split("```",1)[0]
  print('-' * 48, "\n", code , '_' * 48, "\n")
  return code

# 将提示模板与模型和输出解析器组合成一个链
# python_code_chain = python_code_prompt | model | StrOutputParser() | _sanitize_output
python_code_run_chain = python_code_prompt | model | StrOutputParser() | _sanitize_output | PythonREPL().run


In [27]:
python_code_run_chain.invoke({"input":"列出 0-50内的所有质数"})

------------------------------------------------ 
 
def is_prime(n):
    """判断一个数是否为质数"""
    if n <= 1:
        return False
    for i in range(2, int(n**0.5) + 1):
        if n % i == 0:
            return False
    return True

primes = [num for num in range(51) if is_prime(num)]
print(primes)
 ________________________________________________ 



'[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47]\n'